## Problem Title


####  Step 0: The model

Decision Variables:

$$\begin{align*}
&I \in [{1,2,3,4,5,6,7,8,9,10}]\\
&K \in [{1,2,3,4,5,6,7}]\\
&M \in [{1,2,3,4}]\\
\end{align*}$$

$X_{ki} = \begin{cases}
1 & \mbox{if class $k \in K$ is in room $i \in I$,} \\
0 & \mbox{otherwise.}
\end{cases}$

$Z_{kij} = \begin{cases}
1 & \mbox{if class $k \in K$ is moving from room $i \in I$ to room $j \in $I ,} \\
0 & \mbox{otherwise.}
\end{cases}$

$$\begin{array}{rll}
\text{min} & \displaystyle 3Y_1^+ + 2Y_2^+ + 2Y_4^+\\
\text{s.t.}\\
% Goal Optimization Goals
& \displaystyle \sum_{i=1}^I X_{ki}R_{i} +\sum_{i=1}^I\sum_{j=1}^I Z_{kij}TE_{kij} - (Y_{1}^+ - Y_{1}^-) = 1100 \quad \forall k\in K\\
& \displaystyle \sum_{i=1}^I X_{ki}R_{i} - (Y_{2}^+ - Y_{2}^-) = 19 \quad \forall k\in K\\
& \displaystyle \sum_{i=1}^I\sum_{j=1}^I X_{ki}Z_{kij}T_{kij} - (Y_{3}^+ - Y_{3}^-) = 55 \quad \forall k\in K\\
& \displaystyle \sum_{i=1}^I\sum_{j=1}^I X_{ki}Z_{kij}E_{kij} - (Y_{4}^+ - Y_{4}^-) = 19 \quad \forall k\in K\\
% Constraints
& \displaystyle \sum_{i=1}^I\sum_{j=1}^I Z_{kij} \geq 5 \quad \forall k\in K\\
& \displaystyle \sum_{i=9}^I X_{ki} \leq 2 \quad \forall k\in K\\
& \displaystyle \sum_{i=1}^I X_{ki} = 1 \quad \forall k\in K\\
& \displaystyle X_{k8} = 1 \quad \forall k\in K\\
\end{array}$$

$$\begin{align*}
&T_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&E_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I,\text{and } i \neq j \\
&TE_{kij} \geq 0 \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&R_{i} \geq 0 \text{ and integer where } k \in K, i \in I \\
&X_{ki} \in [{0,1}] \text{ where } k \in K, i \in I \\
&Z_{kij} \in [{0,1}] \text{ and integer where } k \in K, i \in I, j \in I, \text{and } i \neq j \\
&Y_{m}^+,Y_{m}^- \geq 0 \text{ where } m \in M, \text{and } Y_{m}^+ \text{ is a surplus variable and } Y_{m}^- \text{is a slack variable }\\
\end{align*}$$

#### Step 1: Import gurobipy module

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

#### Step 1.1 Reading from a file

In [ ]:
MainFile = pd.read_json("Testing.txt",orient='columns')
MainFile

dataframe = pd.DataFrame(MainFile)
print(dataframe)
 
# print(dataframe['Average Transition Time Matrix']) 
# print(dataframe['Transition Risk Exposure Score']) 
# print(dataframe['Room Risk Exposure Score']) 

# dataframe['Average Transition Time Matrix'][0][0] # grabbing values from matrix ['Average Transition Time Matrix'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Transition Risk Exposure Score'][0][0] # grabbing values from matrix ['Transition Risk Exposure Score'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Room Risk Exposure Score']['Score'][0] # grabbing values from matrix ['Room Risk Exposure Score']['Score'][i] where i={0-9}


#### Step 2: Define the model

In [ ]:
m = gp.Model()

#### Step 3: Define your sets

In [ ]:
# number of rooms in the system
I=10
# number of classes in the system
K=7

#### Step 4: Define the parameters

In [ ]:
# dataframe['Average Transition Time Matrix'][0][0] # grabbing values from matrix ['Average Transition Time Matrix'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Transition Risk Exposure Score'][0][0] # grabbing values from matrix ['Transition Risk Exposure Score'][i][j] at (row=i,column=j) where i={0-9} and j={0-9}
# dataframe['Room Risk Exposure Score']['Score'][0] # grabbing values from matrix ['Room Risk Exposure Score']['Score'][i] where i={0-9}

#### Step 5: Define the decision variables

In [ ]:
x={}
for i in range(I):
    for j in range(I):
       

z={}
for i in range(I):
    


#### Step 6: Set the objective function


In [ ]:
m.modelSense = GRB.MINIMIZE

#### Step 6: Add the constraints

In [ ]:
m.addConstrs() 
m.addConstrs()
m.addConstrs() 
m.addConstrs()
m.addConstrs() 
m.addConstrs()
m.addConstrs() 
m.addConstrs()
m.update()

#### Step 7: Solve the model

In [ ]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

#### Step 8: Print variable values  (The Messy Way)

In [ ]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
print('\n')
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.z))    
    

#### Step 8 Alternate: Print the solution (The Easy To Read Way)

#### Optional Step: Exporting the LP File for Debugging
Export it to an LP file and check if it is in the desired shape.

In [ ]:
m.write("checkModel.lp")